#### Imports

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime
import math
import sklearn
from sklearn.model_selection import TimeSeriesSplit

Import and examine datasets

In [2]:
dataset = pd.read_csv('data/all_exchanges_data_cleaned.csv')
fourday = pd.read_csv('data/fourday.csv')

In [3]:
dataset.describe()

,Open,High,Low,Close,Adj Close,Volume,Percent Change
count,19958.000000,19958.000000,19958.000000,19958.000000,19958.000000,1.995800e+04,19958.000000
mean,12359.002648,12432.552746,12275.562052,12356.404874,12356.404874,4.463857e+08,0.018235
std,8513.927306,8558.398208,8460.504323,8510.861717,8510.861717,7.229008e+08,1.368694
min,427.600006,436.519989,419.489990,419.950012,419.950012,0.000000e+00,-12.926546
25%,1085.022461,1090.065003,1081.955017,1086.654968,1086.654968,1.347000e+05,-0.597822
50%,11933.995117,12011.040039,11834.734863,11932.549805,11932.549805,2.048604e+08,0.049819
75%,19283.262696,19401.442383,19161.534668,19287.947266,19287.947266,3.603706e+08,0.665370
max,33335.480469,33484.078125,32897.039063,33154.121094,33154.121094,9.799120e+09,14.347069


In [4]:
fourday.columns

Index(['index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close',
       'Percent Change', 'Index'],
      dtype='object')

In [5]:
fourday.head(5)

,index,Date,Open,High,Low,Close,Adj Close,Percent Change,Index
0,3,2000-10-11,10521.070313,10566.580078,10350.889648,10413.790039,10413.790039,-1.050990,Dow Jones Industrial
1,4,2000-10-11,1009.489990,1009.489990,1009.489990,1009.489990,1009.489990,-2.605886,Euronext100
2,5,2000-10-11,15376.620117,15376.620117,15073.950195,15127.000000,15127.000000,-2.745965,Hang Seng
3,6,2000-10-11,15795.150391,15795.150391,15424.709961,15513.570313,15513.570313,-1.984805,Nikkei 225
4,7,2000-10-12,10424.139648,10460.400391,10023.490234,10034.580078,10034.580078,-3.641421,Dow Jones Industrial


Index Column is redundant and Date column no longer necessary because data is sorted, and will intefere with test-train splits.

In [6]:
fourday.drop(['index', 'Date'], 1, inplace=True)

Split merged dataset by indeces using the Index Column created when wrangling.

In [7]:
DJI = fourday.loc[fourday['Index'] == 'Dow Jones Industrial']
N100 = fourday.loc[fourday['Index'] == 'Euronext100']
HSI = fourday.loc[fourday['Index'] == 'Hang Seng']
N225 = fourday.loc[fourday['Index'] == 'Nikkei 225']

In [8]:
DJI = DJI.reset_index(drop=True)
HSI = HSI.reset_index(drop=True)
N100 = N100.reset_index(drop=True)
N225 = N225.reset_index(drop=True)

For test train split, Index Column is a redundant string variable since datasets are already separated by index. The variable was derived from separate datasets anyways to keep track during the merge. Drop Index Column

In [9]:
DJI.drop('Index', 1, inplace=True)
HSI.drop('Index', 1, inplace=True)
N100.drop('Index', 1, inplace=True)
N225.drop('Index', 1, inplace=True)

Create a second set of datasets where DJI and HSI are shifted a day earlier than the Asian market counterparts. This is in line with NYSE and Euronext exchanges operating at earlier times than Asian markets

In [10]:
DJI_shifted = DJI.drop(DJI.index[0])
HSI_shifted = HSI.drop(DJI.index[0])
N100_shifted = N100.drop(DJI.index[-1])
N225_shifted = N225.drop(DJI.index[-1])

DJI_shifted = DJI_shifted.reset_index(drop=True)
HSI_shifted = HSI_shifted.reset_index(drop=True)
N100_shifted = N100_shifted.reset_index(drop=True)
N225_shifted = N225_shifted.reset_index(drop=True)

In [11]:
DJI.dtypes, DJI_shifted.dtypes

(Open              float64
 High              float64
 Low               float64
 Close             float64
 Adj Close         float64
 Percent Change    float64
 dtype: object,
 Open              float64
 High              float64
 Low               float64
 Close             float64
 Adj Close         float64
 Percent Change    float64
 dtype: object)

In [12]:
print(DJI.shape, HSI.shape, N100.shape, N225.shape, DJI_shifted.shape, HSI_shifted.shape, N100_shifted.shape, N225_shifted.shape)

(4515, 6) (4515, 6) (4515, 6) (4515, 6) (4514, 6) (4514, 6) (4514, 6) (4514, 6)


Re-merge the separate indices. Do the same with the row-shifted datasets

In [13]:
Merged_df = [DJI, N100, HSI, N225]
Merged_df_shifted = [DJI_shifted, N100_shifted, HSI_shifted, N225_shifted]

In [14]:
DJI.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Percent Change'], dtype='object')

All Indeces/Exchanges need to be merged with a suffix after each column to keep track of the index due to repeated column names during the merge. The same will be done to the shifted dataset.

In [15]:
All_exchanges_mg = Merged_df[0].merge(Merged_df[1], how='inner', suffixes=('_Dow', '_Eur'), left_index=True, right_index=True)

All_exchanges_shifted_mg = Merged_df_shifted[0].merge(Merged_df_shifted[1], how='inner', suffixes=('_Dow', '_Eur'), left_index=True, right_index=True)

In [16]:
for column in (Merged_df[2].columns.values):
    Merged_df[2] = Merged_df[2].rename(columns={column: column + '_HSI'})

In [17]:
for column in (Merged_df_shifted[2].columns.values):
    Merged_df_shifted[2] = Merged_df_shifted[2].rename(columns={column: column + '_HSI'})

In [18]:
for column in (Merged_df[3].columns.values):
    Merged_df[3] = Merged_df[3].rename(columns={column: column + '_NIK'})

In [19]:
for column in (Merged_df_shifted[3].columns.values):
    Merged_df_shifted[3] = Merged_df_shifted[3].rename(columns={column: column + '_NIK'})

In [20]:
All_exchanges_mg = All_exchanges_mg.merge(Merged_df[2], how='inner', left_index=True, right_index=True)

All_exchanges_shifted_mg = All_exchanges_shifted_mg.merge(Merged_df_shifted[2], how='inner', left_index=True, right_index=True)

In [21]:
All_exchanges_mg = All_exchanges_mg.merge(Merged_df[3], how='inner', left_index=True, right_index=True)

All_exchanges_shifted_mg = All_exchanges_shifted_mg.merge(Merged_df_shifted[3], how='inner', left_index=True, right_index=True)

Analyze the resulting datasets

In [22]:
All_exchanges_mg.columns

Index(['Open_Dow', 'High_Dow', 'Low_Dow', 'Close_Dow', 'Adj Close_Dow',
       'Percent Change_Dow', 'Open_Eur', 'High_Eur', 'Low_Eur', 'Close_Eur',
       'Adj Close_Eur', 'Percent Change_Eur', 'Open_HSI', 'High_HSI',
       'Low_HSI', 'Close_HSI', 'Adj Close_HSI', 'Percent Change_HSI',
       'Open_NIK', 'High_NIK', 'Low_NIK', 'Close_NIK', 'Adj Close_NIK',
       'Percent Change_NIK'],
      dtype='object')

In [23]:
All_exchanges_shifted_mg.columns

Index(['Open_Dow', 'High_Dow', 'Low_Dow', 'Close_Dow', 'Adj Close_Dow',
       'Percent Change_Dow', 'Open_Eur', 'High_Eur', 'Low_Eur', 'Close_Eur',
       'Adj Close_Eur', 'Percent Change_Eur', 'Open_HSI', 'High_HSI',
       'Low_HSI', 'Close_HSI', 'Adj Close_HSI', 'Percent Change_HSI',
       'Open_NIK', 'High_NIK', 'Low_NIK', 'Close_NIK', 'Adj Close_NIK',
       'Percent Change_NIK'],
      dtype='object')

In [24]:
All_exchanges_mg.shape, All_exchanges_shifted_mg.shape

((4515, 24), (4514, 24))

In [25]:
print(DJI.columns, len(DJI.columns))

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Percent Change'], dtype='object') 6


Each Exchange contains 6 columns, four of these merged produce 24 columns total. Splits can therefore be done per 6 columns manually.

In [26]:
### Check that the above logic is correct rather than assume
All_exchanges_mg[All_exchanges_mg.columns[:6]]

,Open_Dow,High_Dow,Low_Dow,Close_Dow,Adj Close_Dow,Percent Change_Dow
0,10521.070313,10566.580078,10350.889648,10413.790039,10413.790039,-1.050990
1,10424.139648,10460.400391,10023.490234,10034.580078,10034.580078,-3.641421
2,10031.620117,10208.089844,10014.240234,10192.179688,10192.179688,1.570565
3,10184.780273,10272.089844,10177.759766,10238.799805,10238.799805,0.457411
4,10242.870117,10293.919922,10026.450195,10089.709961,10089.709961,-1.456126
...,...,...,...,...,...,...
4510,27560.240234,27605.599609,27338.089844,27452.660156,27452.660156,-0.476363
4511,27514.640625,28026.330078,27511.060547,27781.699219,27781.699219,1.198569
4512,27825.419922,28162.640625,27825.419922,28148.640625,28148.640625,1.682741
4513,28214.240234,28354.480469,27728.029297,27772.759766,27772.759766,-1.335343


### Test Train Split

In [27]:
### Create initial time series split, assign it to variable
tscv = TimeSeriesSplit(n_splits=10)

Y variable is the percent change, that is what we are aiming to predict, on both datasets

In [30]:
y = All_exchanges_mg[All_exchanges_mg.columns[5]]
y2 = All_exchanges_shifted_mg[All_exchanges_shifted_mg.columns[5]]

X variable will be our Dow Dataset, first six rows of merged data

In [31]:
X = All_exchanges_mg[All_exchanges_mg.columns[6:]]
X2 = All_exchanges_shifted_mg[All_exchanges_shifted_mg.columns[6:]]

Make sure equal number of rows are present before the split

In [32]:
print(X.shape, y.shape, X2.shape, y2.shape)

(4515, 18) (4515,) (4514, 18) (4514,)


Run timeseries test train splits

In [33]:
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]

In [34]:
for train_index, test_index in tscv.split(X2):
    X2_train, X2_test = X2.values[train_index], X2.values[test_index]
    y2_train, y2_test = y2.values[train_index], y2.values[test_index]

Check shapes to make sure splits were accurate

In [35]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X2_train.shape, y2_train.shape, X2_test.shape, y2_test.shape)

(4105, 18) (4105,) (410, 18) (410,) (4104, 18) (4104,) (410, 18) (410,)


Save all splits to csv files

In [36]:
pd.DataFrame(X_train).to_csv('data/X_train.csv', index=False)
pd.DataFrame(X_test).to_csv('data/X_test.csv', index=False)
pd.DataFrame(y_train).to_csv('data/y_train.csv', index=False)
pd.DataFrame(y_test).to_csv('data/y_test.csv', index=False)

pd.DataFrame(X2_train).to_csv('data/X2_train.csv', index=False)
pd.DataFrame(X2_test).to_csv('data/X2_test.csv', index=False)
pd.DataFrame(y2_train).to_csv('data/y2_train.csv', index=False)
pd.DataFrame(y2_test).to_csv('data/y2_test.csv', index=False)

In [ ]:
### Times in Greenwich Mean Time

###Both day ahead
### Japan Exchange Group / N225    12am to 6am
### Hong Kong / Hang Seng =        1:30am to 8am

###Both lag day behind locally
### EuroNext / N100 =              8am to 4:30pm
### NYSE / DJI =                   2:30pm to 9pm

###Observations
### Jap and Hong Kong close before EuroNext opens SAME DAY
### Hong Kong closes RIGHT as Euronext Opens SAME DAY
###New York and EuroNext close before Jap and Hong Kong open THE DAY BEFORE
###NYSE is mid-dayish when EURO closes

###For test case leave out NYSE in test train split and begin running models on that